In [2]:
from helium10Scrapper.path import PROJECT_ROOT
import requests
import json
import pandas as pd
import time
from pathlib import Path
import os

ModuleNotFoundError: No module named 'requests'

In [3]:
def create_black_box_query(the_filter):
    r = requests.post(
        f"https://research-tools.helium10.com/api/blackbox/v1/search/products?accountId={ACCOUNT_ID}",
        json=the_filter,
        headers={"Authorization": f"Bearer {BEARER_TOKEN}"},
    )

    black_box_query_creation_response = {}
    if r.status_code == 201:
        black_box_query_creation_response = r.json()
    else:
        raise Exception(
            f"Error occured when creating black box query, status_code: {r.status_code}"
        )

    return black_box_query_creation_response


def get_black_box_query_result(black_box_query_id):
    r = requests.get(
        f"https://research-tools.helium10.com/api/blackbox/v1/search/products/{black_box_query_id}/results?",
        params={"accountId": ACCOUNT_ID, "page": 1, "per_page": 1000},
        headers={"Authorization": f"Bearer {BEARER_TOKEN}"},
    )
    black_box_result_dict = r.json()
    return black_box_result_dict


def create_cerebro_query(product_asin):
    cerebro_create_query_endpoint = (
        f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single"
    )
    params = {
        "accountId": ACCOUNT_ID,
    }
    body = {
        "marketplace": "ATVPDKIKX0DER",
        "productId": product_asin,
        "adminSearch": False,
        "exactProduct": False,
    }
    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    r = requests.post(
        url=cerebro_create_query_endpoint, params=params, headers=headers, json=body
    )
    return r


def get_cerebro_query_result(cerebro_query_id):
    cerebro_get_query_endpoint = f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single/{cerebro_query_id}/data"
    params = {
        "accountId": ACCOUNT_ID,
        "include-all": 0,
        "include-any": 1,
        "page": 1,
        "per_page": 1000,
        "sort": "-iq",
    }

    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    r = requests.get(cerebro_get_query_endpoint, params=params, headers=headers)
    return r

In [4]:
PROJECT_ROOT = Path(os.getcwd())
MODULE_ROOT = PROJECT_ROOT.joinpath("helium10scrapper")
HEADERS_DIR = PROJECT_ROOT.joinpath("headers")

NameError: name 'Path' is not defined

In [ ]:
BATCH_ID = "9999"
# get filter from pre-written json file
f = open(PROJECT_ROOT.joinpath("black_box_filter.01.json"))
all_cat_filter = json.load(f)
f.close()

In [5]:
BEARER_TOKEN = 
ACCOUNT_ID = 1545862407

print("create black box query...")
black_box_query_creation_response = create_black_box_query(all_cat_filter)



SyntaxError: invalid syntax (1849302398.py, line 1)

In [6]:
# parse black box query_id
print("parse black box query...")

black_box_query_id = ""
try:
    black_box_query_id = black_box_query_creation_response["data"]["id"]
except KeyError:
    raise (
        "No id found in response, error may occrued when creating black box query"
    )

parse black box query...


NameError: name 'black_box_query_creation_response' is not defined

In [7]:
# get black box query result
print("get black box result")
black_box_query_id = black_box_query_creation_response["data"]["id"]
black_box_result_dict = get_black_box_query_result(
    black_box_query_id=black_box_query_id
)

black_box_result_data = []
try:
    black_box_result_data = black_box_result_dict["data"]["tableData"]
except KeyError:
    raise (
        "Can't find table data, Error occrued when requesting for black box query result"
    )

get black box result


NameError: name 'black_box_query_creation_response' is not defined

In [8]:
print("dump black box result")
data = []
for data_dict in black_box_result_data:
    try:
        data.append([black_box_query_id, data_dict["asin"], data_dict["id"]])
    except KeyError:
        continue

black_box_result_tb = pd.DataFrame(data, columns=["query_id", "asin", "id"])
black_box_result_tb.to_csv(
    PROJECT_ROOT.joinpath(f"black_box_result.{BATCH_ID}.csv"), index=False
)

dump black box result


NameError: name 'black_box_result_data' is not defined

In [9]:
# create cerebro query for each asin in black_box_result_data
print("create cerebro query result")
cerebro_query_id_dict = {}
for product in black_box_result_data[:10]:
    asin = product["asin"]
    print(asin, end="..., ")
    r = create_cerebro_query(asin)
    print(r.status_code, end=", sleeping...\n")
    cerebro_query_id_dict[asin] = r
    time.sleep(5)

# dump cerebro result as json
print("dump cerebro query response (as json)")
output_tmp_dict = {}
for asin, response in cerebro_query_id_dict.items():
    output_tmp_dict[asin] = response.json()

f = open(PROJECT_ROOT.joinpath(f"cerebro_query_id_dict.{BATCH_ID}.json"), "w")
json.dump(output_tmp_dict, f, indent=4)
f.close()

create cerebro query result


NameError: name 'black_box_result_data' is not defined

In [10]:
# create cerebro query for each asin in black_box_result_data
print("get cerebro query result")
cerebro_result_dict = {}
for asin, response in cerebro_query_id_dict.items():
    print(asin, end="\r")
    cerebro_query_id = ""
    try:
        cerebro_query_id = response.json()["data"]["id"]
    except KeyError:
        print("no result for this asin")
        continue

    cerebro_result_dict[asin] = get_cerebro_query_result(cerebro_query_id)
    print(cerebro_result_dict[asin].status_code, end="\r")
    time.sleep(5)


get cerebro query result


In [13]:
print("formatting data")
data = []
for asin, response in cerebro_result_dict.items():
    serialized_response = response.json()
    try:
        search_id = serialized_response["data"]["searchId"]
        for organic_rank, record in enumerate(
            serialized_response["data"]["tableData"][:20]
        ):
            data.append(
                [
                    asin,
                    search_id,
                    organic_rank + 1,
                    record["organicPosition"],
                    record["phraseId"],
                    record["phrase"],
                ]
            )
    except KeyError as e:
        print(str(e))
        continue

formatting data


In [15]:
print("dump cerebro result (as csv)")
cerebro_result_tb = pd.DataFrame(
    data,
    columns=[
        "asin",
        "search_id",
        "organic_rank",
        "organic_position",
        "phrase_id",
        "phrase",
    ],
)
cerebro_result_tb.to_csv(
    PROJECT_ROOT.joinpath(f"cerebro_result.{BATCH_ID}.csv"), index=False
)

# dump as json
print("dump cerebro result (as json)")
output_tmp_dict = {}
for asin, response in cerebro_query_id_dict.items():
    output_tmp_dict["asin"] = response.json()

f = open(PROJECT_ROOT.joinpath(f"cerebro_query_result.{BATCH_ID}.json"), "w")
json.dump(output_tmp_dict, f, indent=4)
f.close()


dump cerebro result (as csv)


NameError: name 'pd' is not defined

In [ ]:

black_box_query_creation_response["data"]




In [ ]:
black_box_query_creation_response["data"]["id"]

In [ ]:
black_box_result_dict["data"]["tableData"]

In [ ]:
black_box_result_data[:10]



In [ ]:
cerebro_query_id_dict

In [16]:
for product in black_box_result_data[:3]:
        asin = product["asin"]
        print(asin, end="..., ")
        r = create_cerebro_query(asin)
        print(r.status_code, end=", sleeping...\n")
        cerebro_query_id_dict[asin] = r
        print(r)
        print("fewefwe")
        time.sleep(5)

NameError: name 'black_box_result_data' is not defined

In [ ]:
cerebro_query_id_dict.items()

output_tmp_dict[B00M9NW31A]

In [ ]:
output_tmp_dict['B076PQ2KC9']

In [ ]:
for asin, response in cerebro_query_id_dict.items():
        print(asin, end="\r")
        print(response)

In [ ]:
cerebro_result_dict.keys()

In [ ]:
data[:][2]

In [ ]:
cerebro_result_dict.items()

In [ ]:
#asin_and_boxid=[]
#len(black_box_result_dict['data']['tableData'])
#for i in range (len(black_box_result_dict['data']['tableData'])):
 #   asin_and_boxid.append ((black_box_result_dict['data']['tableData'][i]['id'],black_box_result_dict['data']['tableData'][i]['asin']))

In [18]:
from helium10Scrapper.path import PROJECT_ROOT
import requests
import json
import pandas as pd
import time



BEARER_TOKEN = env.BEARER_TOKEN
ACCOUNT_ID = env.ID


def create_black_box_query(the_filter):
    r = requests.post(
        f"https://research-tools.helium10.com/api/blackbox/v1/search/products?accountId={ACCOUNT_ID}",
        json=the_filter,  #the filter是black box filter 01.json
        headers={"Authorization": f"Bearer {BEARER_TOKEN}"},
    )

    black_box_query_creation_response = {}
    if r.status_code == 201:
        black_box_query_creation_response = r.json()
    else:
        raise Exception(
            f"Error occured when creating black box query, status_code: {r.status_code}"
        )

    return black_box_query_creation_response


def get_black_box_query_result(black_box_query_id):
    r = requests.get(
        f"https://research-tools.helium10.com/api/blackbox/v1/search/products/{black_box_query_id}/results?",
        params={"accountId": ACCOUNT_ID, "page": 1, "per_page": 1000},
        headers={"Authorization": f"Bearer {BEARER_TOKEN}"},
    )
    black_box_result_dict = r.json()
    return black_box_result_dict


def create_cerebro_query(product_asin):
    cerebro_create_query_endpoint = (
        f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single"
    )
    params = {
        "accountId": ACCOUNT_ID,
    }
    body = {
        "marketplace": "ATVPDKIKX0DER",
        "productId": product_asin,
        "adminSearch": False,
        "exactProduct": False,
    }
    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    r = requests.post(
        url=cerebro_create_query_endpoint, params=params, headers=headers, json=body
    )
    return r


def get_cerebro_query_result(cerebro_query_id):
    cerebro_get_query_endpoint = f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single/{cerebro_query_id}/data"
    
    params = {
        "accountId": ACCOUNT_ID,
        "include-all": 0,
        "include-any": 1,
        "page": 1,
        "per_page": 1000,
        "sort": "-iq",
    }

    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    r = requests.get(cerebro_get_query_endpoint, params=params, headers=headers)
    return r


if __name__ == "__main__":
    BATCH_ID = "9999"
    # get filter from pre-written json file
    f = open(PROJECT_ROOT.joinpath("black_box_filter.01.json"))
    all_cat_filter = json.load(f)
    f.close()

    # create black box query
    print("create black box query...")
    black_box_query_creation_response = create_black_box_query(all_cat_filter)  #response裡面是字典裝字典 第二個字典裡項目是key 值是資料內容

    # parse black box query_id
    print("parse black box query...")

    black_box_query_id = ""
    try:
        black_box_query_id = black_box_query_creation_response["data"]["id"]
    except KeyError:
        raise (
            "No id found in response, error may occrued when creating black box query"
        )

    # get black box query result
    print("get black box result")
    black_box_query_id = black_box_query_creation_response["data"]["id"]
    black_box_result_dict = get_black_box_query_result(
        black_box_query_id=black_box_query_id  #***********************
    )

    black_box_result_data = []
    try:
        black_box_result_data = black_box_result_dict["data"]["tableData"]   #有一大堆商品，然後每個商品都有id跟asin。
    except KeyError:
        raise (
            "Can't find table data, Error occrued when requesting for black box query result"
        )

    # dump resutl to csv
    print("dump black box result")
    data = []
    for data_dict in black_box_result_data:
        try:
            data.append([black_box_query_id, data_dict["asin"], data_dict["id"]])
        except KeyError:
            continue

    black_box_result_tb = pd.DataFrame(data, columns=["query_id", "asin", "id"])
    black_box_result_tb.to_csv(
        PROJECT_ROOT.joinpath(f"black_box_result.{BATCH_ID}.csv"), index=False
    )

    # create cerebro query for each asin in black_box_result_data
    print("create cerebro query result")
    cerebro_query_id_dict = {}
    for product in black_box_result_data[:10]:
        asin = product["asin"]
        print(asin, end="..., ")
        r = create_cerebro_query(asin)
        print(r.status_code, end=", sleeping...\n")
        cerebro_query_id_dict[asin] = r 
        time.sleep(5)

    # dump cerebro result as json
    print("dump cerebro query response (as json)")
    output_tmp_dict = {}
    for asin, response in cerebro_query_id_dict.items():
        output_tmp_dict[asin] = response.json()

    f = open(PROJECT_ROOT.joinpath(f"cerebro_query_id_dict.{BATCH_ID}.json"), "w")
    json.dump(output_tmp_dict, f, indent=4)
    f.close()

    # get cerebro query result
    print("get cerebro query result")
    cerebro_result_dict = {}
    cerebro_query_id_list = [] ################
    for asin, response in cerebro_query_id_dict.items():
        print(asin, end="\r")
        cerebro_query_id = ""
        
        try:
            cerebro_query_id = response.json()["data"]["id"]
            cerebro_query_id_list.append(cerebro_query_id) ###############
        except KeyError:
            print("no result for this asin")
            continue

        cerebro_result_dict[asin] = get_cerebro_query_result(cerebro_query_id)
        print(cerebro_result_dict[asin].status_code, end="\r")
        time.sleep(5)

    box_id=[]
    for i in range (len(black_box_result_dict['data']['tableData'])):
        box_id.append (black_box_result_dict['data']['tableData'][i]['id'])


    # formatting data
    print("formatting data")
    data = []
    ordinal_number_asin = 0
    for asin, response in cerebro_result_dict.items():
        serialized_response = response.json()
        try:
            search_id = serialized_response["data"]["searchId"]
            for organic_rank, record in enumerate(
                serialized_response["data"]["tableData"][:20]
            ):
                data.append(
                    [
                        box_id[ordinal_number_asin],
                        asin,
                        search_id,
                        organic_rank + 1,
                        record["organicPosition"],
                        record["phraseId"],
                        record["phrase"],
                    ]
                )
        except KeyError as e:
            print(str(e))
            continue
    # dump as csv
    print("dump cerebro result (as csv)")
    cerebro_result_tb = pd.DataFrame(
        data,
        columns=[
            "asin",
            "search_id",
            "organic_rank",
            "organic_position",
            "phrase_id",
            "phrase",
        ],
    )
    cerebro_result_tb.to_csv(
        PROJECT_ROOT.joinpath(f"cerebro_result.{BATCH_ID}.csv"), index=False
    )

    # dump as json
    print("dump cerebro result (as json)")
    output_tmp_dict = {}
    for asin, response in cerebro_query_id_dict.items():
        output_tmp_dict["asin"] = response.json()

    f = open(PROJECT_ROOT.joinpath(f"cerebro_query_result.{BATCH_ID}.json"), "w")
    json.dump(output_tmp_dict, f, indent=4)
    f.close()


ModuleNotFoundError: No module named 'requests'

In [ ]:
cerebro_query_id_list

In [ ]:
cerebro_query_id_dict.items()

In [ ]:
#match 實驗用的id list跟asin id組合的長度。 (200個)
for i in range(190):
    cerebro_query_id_list.append(i) 

cerebro_query_id_list

In [ ]:
for i in range(len(data)):
    data[i].insert(0,box_id[i])

In [19]:
    # formatting data
    print("formatting data")
    data = []
    ordinal_number_asin = 0
    for asin, response in cerebro_result_dict.items():
        serialized_response = response.json()
        ordinal_number_asin = ordinal_number_asin+1
        try:
            search_id = serialized_response["data"]["searchId"]
            for organic_rank, record in enumerate(
                serialized_response["data"]["tableData"][:20]
            ):
                data.append(
                    [
                        box_id[ordinal_number_asin],
                        asin,
                        search_id,
                        organic_rank + 1,
                        record["organicPosition"],
                        record["phraseId"],
                        record["phrase"],
                    ]
                )
        except KeyError as e:
            print(str(e))
            continue

formatting data


In [20]:
data

[]

In [21]:
cerebro_result_dict['B00AY059ZC'].json()

KeyError: 'B00AY059ZC'

In [22]:
for i in range(len(black_box_result_tb['asin'])):
    for j in range(len(cerebro_result_dict[asin].json()['data']['tableData'])):
        cerebro_result_dict[asin].json()['data']['tableData'][j]['cerebro_query_id']= cerebro_query_id_list[i]
        cerebro_result_dict[asin].json()['data']['tableData'][j]['black_box_query_asin']=black_box_result_tb['asin'][i]
        cerebro_result_dict[asin].json()['data']['tableData'][j]['black_box_query_id'] =black_box_query_id

NameError: name 'black_box_result_tb' is not defined

In [ ]:
for rowidx, row in black_box_result_tb.iterrows():
    print(rowidx,row['asin'])

In [ ]:
black_box_result_tb 

In [ ]:
black_box_query_creation_response["data"]["id"]

In [ ]:
#c id的list
cerebro_query_id_list

In [ ]:
##asin的list
cerebro_result_dict.keys()

In [ ]:
final_data = []      
for black_box_asin in cerebro_result_dict.keys() : 
        final_data = cerebro_result_dict[black_box_asin].json()['data']['tableData'].copy()
        for i in range(len(final_data)):
                final_data[i]['black_box_query_id'] =black_box_query_id #filter 01 的 black box id 
                final_data[i]['black_box_query_asin']=black_box_asin
                final_data[i]['cerebro_query_id']= cerebro_query_id_dict[black_box_asin].json()['data']['id']
final_data



In [23]:
#formatting data to fit airtable api requiement
airtable_data = {'records':[]}
for i in range(len(final_data)):
    airtable_data['records'].append({'fields':{
    'black_box_query_id':'tempId',
    'black_box_query_asin':'tempAsin',
    'cerebro_query_id':'tempCerebroId',
    'phraseId':'tempPhraseId',
    'sponsoredPosition':'tempSponsoredPosition',
    'sponsoredScrapedAt':'tempSponsoredScrapedAt',
    'matchType':'tempMatchType',
    'organicPosition':'tempOrganicPosition',
    'organicScrapedAt':'tempOrganicScrapedAt',
    'phrase':'tempPhrase',
    'resultsNumber':'tempResultsNumber',
    'resultsNumberOver':'tempResultsNumberOver',
    'resultsNumberUpdatedAt':'tempResultsNumberUpdatedAt',
    'impressionExact30':'tempImpressionExact30',
    'exactTitleMatchProductsCount':'tempExactTitleMatchProductsCount',
    'iq':'tempIq',
    'sponsoredAsins':'tempSponsoredAsins',
    'amazonChoice':'tempAmazonChoice',
    'currentAsin':'tempCurrentAsin',
    'amzSuggestedPosition':'tempAmzSuggestedPosition',
    'amzSuggestedScrapedAt':'tempAmzSuggestedScrapedAt',
    'searchVolumeTrend30':'tempSearchVolumeTrend30',
    'newCprExact':'tempNewCprExact',
    'newCprBroad':'tempNewCprBroad',
    'monthlySales':'tempMonthlySales',
    'clickShareRate':'tempClickShareRate',
    'conversionShareRate':'tempConversionShareRate',
    'cpc':'tempCpc',
    'highCpc': None,
    'lowCpc': None,
    'searchFrequencyRank': None,
    'octoberSeasonalSearchVolume': None,
    'decemberSeasonalSearchVolume': None,
    'novemberSeasonalSearchVolume': None
    },
    "id":i
    })
  
    
    airtable_data['records'][i]['fields']['black_box_query_id']=final_data[i]['black_box_query_id']
    airtable_data['records'][i]['fields']['black_box_query_asin']=final_data[i]['black_box_query_asin']
    airtable_data['records'][i]['fields']['cerebro_query_id']=final_data[i]['cerebro_query_id']
    airtable_data['records'][i]['fields']['phraseId']=final_data[i]['phraseId']
    airtable_data['records'][i]['fields']['sponsoredPosition']=final_data[i]['sponsoredPosition']
    airtable_data['records'][i]['fields']['matchType']=final_data[i]['matchType']
    airtable_data['records'][i]['fields']['organicPosition']=final_data[i]['organicPosition']
    airtable_data['records'][i]['fields']['phrase']=final_data[i]['phrase']
    airtable_data['records'][i]['fields']['resultsNumber']=final_data[i]['resultsNumber']
    airtable_data['records'][i]['fields']['resultsNumberOver']=final_data[i]['resultsNumberOver']
    airtable_data['records'][i]['fields']['resultsNumberUpdatedAt']=final_data[i]['resultsNumberUpdatedAt']
    airtable_data['records'][i]['fields']['impressionExact30']=final_data[i]['impressionExact30']
    airtable_data['records'][i]['fields']['exactTitleMatchProductsCount']=final_data[i]['exactTitleMatchProductsCount']
    airtable_data['records'][i]['fields']['iq']=final_data[i]['iq']
    airtable_data['records'][i]['fields']['sponsoredAsins']=final_data[i]['sponsoredAsins']
    airtable_data['records'][i]['fields']['amazonChoice']=final_data[i]['amazonChoice']
    airtable_data['records'][i]['fields']['currentAsin']=final_data[i]['currentAsin']
    airtable_data['records'][i]['fields']['amzSuggestedPosition']=final_data[i]['amzSuggestedPosition']
    airtable_data['records'][i]['fields']['amzSuggestedScrapedAt']=final_data[i]['amzSuggestedScrapedAt']
    airtable_data['records'][i]['fields']['searchVolumeTrend30']=final_data[i]['searchVolumeTrend30']
    airtable_data['records'][i]['fields']['newCprExact']=final_data[i]['newCprExact']
    airtable_data['records'][i]['fields']['newCprBroad']=final_data[i]['newCprBroad']
    airtable_data['records'][i]['fields']['monthlySales']=final_data[i]['monthlySales']
    airtable_data['records'][i]['fields']['clickShareRate']=final_data[i]['clickShareRate']
    airtable_data['records'][i]['fields']['conversionShareRate']=final_data[i]['conversionShareRate']
    airtable_data['records'][i]['fields']['cpc']=final_data[i]['cpc']
    airtable_data['records'][i]['fields']['highCpc']=final_data[i]['highCpc']
    airtable_data['records'][i]['fields']['lowCpc']=final_data[i]['lowCpc']
    airtable_data['records'][i]['fields']['searchFrequencyRank']=final_data[i]['searchFrequencyRank']
    airtable_data['records'][i]['fields']['octoberSeasonalSearchVolume']=final_data[i]['octoberSeasonalSearchVolume']
    airtable_data['records'][i]['fields']['decemberSeasonalSearchVolume']=final_data[i]['decemberSeasonalSearchVolume']
    airtable_data['records'][i]['fields']['novemberSeasonalSearchVolume']=final_data[i]['novemberSeasonalSearchVolume']
    
    

NameError: name 'final_data' is not defined

In [ ]:
dict0i={}
for i in [0,1,2]:
    dict0i["dwd"]:'ewfef'

In [ ]:
dict0i.values()

In [ ]:
airtable_data[records].append({'a':'ewef'})

In [ ]:
airtable_data

In [ ]:
from helium10Scrapper.path import PROJECT_ROOT
import requests
import json
import pandas as pd
import time



BEARER_TOKEN = env.BEARER_TOKEN

def create_black_box_query(the_filter):
    r = requests.post(
        f"https://research-tools.helium10.com/api/blackbox/v1/search/products?accountId={ACCOUNT_ID}",
        json=the_filter,  #the filter是black box filter 01.json
        headers={"Authorization": f"Bearer {BEARER_TOKEN}"},
    )

    black_box_query_creation_response = {}
    if r.status_code == 201:
        black_box_query_creation_response = r.json()
    else:
        raise Exception(
            f"Error occured when creating black box query, status_code: {r.status_code}"
        )

    return black_box_query_creation_response


def get_black_box_query_result(black_box_query_id):
    r = requests.get(
        f"https://research-tools.helium10.com/api/blackbox/v1/search/products/{black_box_query_id}/results?",
        params={"accountId": ACCOUNT_ID, "page": 1, "per_page": 1000},
        headers={"Authorization": f"Bearer {BEARER_TOKEN}"},
    )
    black_box_result_dict = r.json()
    return black_box_result_dict


def create_cerebro_query(product_asin):
    cerebro_create_query_endpoint = (
        f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single"
    )
    params = {
        "accountId": ACCOUNT_ID,
    }
    body = {
        "marketplace": "ATVPDKIKX0DER",
        "productId": product_asin,
        "adminSearch": False,
        "exactProduct": False,
    }
    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    r = requests.post(
        url=cerebro_create_query_endpoint, params=params, headers=headers, json=body
    )
    return r


def get_cerebro_query_result(cerebro_query_id):
    cerebro_get_query_endpoint = f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single/{cerebro_query_id}/data"
    
    params = {
        "accountId": ACCOUNT_ID,
        "include-all": 0,
        "include-any": 1,
        "page": 1,
        "per_page": 1000,
        "sort": "-iq",
    }

    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    r = requests.get(cerebro_get_query_endpoint, params=params, headers=headers)
    return r


if __name__ == "__main__":
    BATCH_ID = "9999"
    # get filter from pre-written json file
    f = open(PROJECT_ROOT.joinpath("black_box_filter.01.json"))
    all_cat_filter = json.load(f)
    f.close()

    # create black box query
    print("create black box query...")
    black_box_query_creation_response = create_black_box_query(all_cat_filter)  #response裡面是字典裝字典 第二個字典裡項目是key 值是資料內容

    # parse black box query_id
    print("parse black box query...")

    black_box_query_id = ""
    try:
        black_box_query_id = black_box_query_creation_response["data"]["id"]
    except KeyError:
        raise (
            "No id found in response, error may occrued when creating black box query"
        )

    # get black box query result
    print("get black box result")
    black_box_query_id = black_box_query_creation_response["data"]["id"]
    black_box_result_dict = get_black_box_query_result(
        black_box_query_id=black_box_query_id  #***********************
    )

    black_box_result_data = []
    try:
        black_box_result_data = black_box_result_dict["data"]["tableData"]   #有一大堆商品，然後每個商品都有id跟asin。
    except KeyError:
        raise (
            "Can't find table data, Error occrued when requesting for black box query result"
        )

    # dump resutl to csv
    print("dump black box result")
    data = []
    for data_dict in black_box_result_data:
        try:
            data.append([black_box_query_id, data_dict["asin"], data_dict["id"]])
        except KeyError:
            continue

    black_box_result_tb = pd.DataFrame(data, columns=["query_id", "asin", "id"])
    black_box_result_tb.to_csv(
        PROJECT_ROOT.joinpath(f"black_box_result.{BATCH_ID}.csv"), index=False
    )

    # create cerebro query for each asin in black_box_result_data
    print("create cerebro query result")
    cerebro_query_id_dict = {}
    for product in black_box_result_data[:10]:
        asin = product["asin"]
        print(asin, end="..., ")
        r = create_cerebro_query(asin)
        print(r.status_code, end=", sleeping...\n")
        cerebro_query_id_dict[asin] = r 
        time.sleep(5)

    # dump cerebro result as json
    print("dump cerebro query response (as json)")
    output_tmp_dict = {}
    for asin, response in cerebro_query_id_dict.items():
        output_tmp_dict[asin] = response.json()

    f = open(PROJECT_ROOT.joinpath(f"cerebro_query_id_dict.{BATCH_ID}.json"), "w")
    json.dump(output_tmp_dict, f, indent=4)
    f.close()

    # get cerebro query result
    print("get cerebro query result")
    cerebro_result_dict = {}
    cerebro_query_id_list = [] ################ new
    for asin, response in cerebro_query_id_dict.items():
        print(asin, end="\r")
        cerebro_query_id = ""
        
        try:
            cerebro_query_id = response.json()["data"]["id"]
            cerebro_query_id_list.append(cerebro_query_id) ############### new
        except KeyError:
            print("no result for this asin")
            continue

        cerebro_result_dict[asin] = get_cerebro_query_result(cerebro_query_id)
        print(cerebro_result_dict[asin].status_code, end="\r")
        time.sleep(5)

    box_id=[]
    for i in range (len(black_box_result_dict['data']['tableData'])):
        box_id.append (black_box_result_dict['data']['tableData'][i]['id'])


    # merging data
    final_data = []      
    for black_box_asin in cerebro_result_dict.keys() : 
        final_data = cerebro_result_dict[black_box_asin].json()['data']['tableData'].copy()
        for i in range(len(final_data)):
            final_data[i]['black_box_query_id'] =black_box_query_id #filter 01 的 black box id 
            final_data[i]['black_box_query_asin']=black_box_asin
            final_data[i]['cerebro_query_id']= cerebro_query_id_dict[black_box_asin].json()['data']['id']

    
#formatting data to fit airtable api requiement
airtable_data = {'records':[]}
for i in range(len(final_data)):
    airtable_data['records'].append({'fields':{
    'black_box_query_id':'tempId',
    'black_box_query_asin':'tempAsin',
    'cerebro_query_id':'tempCerebroId',
    'phraseId':'tempPhraseId',
    'sponsoredPosition':'tempSponsoredPosition',
    'sponsoredScrapedAt':'tempSponsoredScrapedAt',
    'matchType':'tempMatchType',
    'organicPosition':'tempOrganicPosition',
    'organicScrapedAt':'tempOrganicScrapedAt',
    'phrase':'tempPhrase',
    'resultsNumber':'tempResultsNumber',
    'resultsNumberOver':'tempResultsNumberOver',
    'resultsNumberUpdatedAt':'tempResultsNumberUpdatedAt',
    'impressionExact30':'tempImpressionExact30',
    'exactTitleMatchProductsCount':'tempExactTitleMatchProductsCount',
    'iq':'tempIq',
    'sponsoredAsins':'tempSponsoredAsins',
    'amazonChoice':'tempAmazonChoice',
    'currentAsin':'tempCurrentAsin',
    'amzSuggestedPosition':'tempAmzSuggestedPosition',
    'amzSuggestedScrapedAt':'tempAmzSuggestedScrapedAt',
    'searchVolumeTrend30':'tempSearchVolumeTrend30',
    'newCprExact':'tempNewCprExact',
    'newCprBroad':'tempNewCprBroad',
    'monthlySales':'tempMonthlySales',
    'clickShareRate':'tempClickShareRate',
    'conversionShareRate':'tempConversionShareRate',
    'cpc':'tempCpc',
    'highCpc': None,
    'lowCpc': None,
    'searchFrequencyRank': None,
    'octoberSeasonalSearchVolume': None,
    'decemberSeasonalSearchVolume': None,
    'novemberSeasonalSearchVolume': None
    },
    "id":i
    })
  
    airtable_data['records'][i]['fields']['black_box_query_id']=final_data[i]['black_box_query_id']
    airtable_data['records'][i]['fields']['black_box_query_asin']=final_data[i]['black_box_query_asin']
    airtable_data['records'][i]['fields']['cerebro_query_id']=final_data[i]['cerebro_query_id']
    airtable_data['records'][i]['fields']['phraseId']=final_data[i]['phraseId']
    airtable_data['records'][i]['fields']['sponsoredPosition']=final_data[i]['sponsoredPosition']
    airtable_data['records'][i]['fields']['matchType']=final_data[i]['matchType']
    airtable_data['records'][i]['fields']['organicPosition']=final_data[i]['organicPosition']
    airtable_data['records'][i]['fields']['phrase']=final_data[i]['phrase']
    airtable_data['records'][i]['fields']['resultsNumber']=final_data[i]['resultsNumber']
    airtable_data['records'][i]['fields']['resultsNumberOver']=final_data[i]['resultsNumberOver']
    airtable_data['records'][i]['fields']['resultsNumberUpdatedAt']=final_data[i]['resultsNumberUpdatedAt']
    airtable_data['records'][i]['fields']['impressionExact30']=final_data[i]['impressionExact30']
    airtable_data['records'][i]['fields']['exactTitleMatchProductsCount']=final_data[i]['exactTitleMatchProductsCount']
    airtable_data['records'][i]['fields']['iq']=final_data[i]['iq']
    airtable_data['records'][i]['fields']['sponsoredAsins']=final_data[i]['sponsoredAsins']
    airtable_data['records'][i]['fields']['amazonChoice']=final_data[i]['amazonChoice']
    airtable_data['records'][i]['fields']['currentAsin']=final_data[i]['currentAsin']
    airtable_data['records'][i]['fields']['amzSuggestedPosition']=final_data[i]['amzSuggestedPosition']
    airtable_data['records'][i]['fields']['amzSuggestedScrapedAt']=final_data[i]['amzSuggestedScrapedAt']
    airtable_data['records'][i]['fields']['searchVolumeTrend30']=final_data[i]['searchVolumeTrend30']
    airtable_data['records'][i]['fields']['newCprExact']=final_data[i]['newCprExact']
    airtable_data['records'][i]['fields']['newCprBroad']=final_data[i]['newCprBroad']
    airtable_data['records'][i]['fields']['monthlySales']=final_data[i]['monthlySales']
    airtable_data['records'][i]['fields']['clickShareRate']=final_data[i]['clickShareRate']
    airtable_data['records'][i]['fields']['conversionShareRate']=final_data[i]['conversionShareRate']
    airtable_data['records'][i]['fields']['cpc']=final_data[i]['cpc']
    airtable_data['records'][i]['fields']['highCpc']=final_data[i]['highCpc']
    airtable_data['records'][i]['fields']['lowCpc']=final_data[i]['lowCpc']
    airtable_data['records'][i]['fields']['searchFrequencyRank']=final_data[i]['searchFrequencyRank']
    airtable_data['records'][i]['fields']['octoberSeasonalSearchVolume']=final_data[i]['octoberSeasonalSearchVolume']
    airtable_data['records'][i]['fields']['decemberSeasonalSearchVolume']=final_data[i]['decemberSeasonalSearchVolume']
    airtable_data['records'][i]['fields']['novemberSeasonalSearchVolume']=final_data[i]['novemberSeasonalSearchVolume']
    
    


In [ ]:
#update data to airtable
airtable_data = {
    "fields":{
        'black_box_query_id' : 12345
    },
    "id":"1215"
}

AIRTABLE_URL = os.getenv("AIRTABLE_URL")
AIRTABLE_HEADER = {
    "Authorization": f"Bearer {os.getenv('AIRTABLE_TOKEN')}",
    "Content-Type": "application/json"
}
r = requests.post(url, headers=headers, json = airtable_data)
r.json()

if r.status_code == 201: 
    print("data has been succesfully uploaded")
else:
    raise Exception(
        f"Error occured when uploading, status_code: {r.status_code}"
    )
r = requests.post(url, headers=headers, json = airtable_data)
r.json()

In [ ]:
f = open(PROJECT_ROOT.joinpath("cerebro_query_result.9999.json"))
airtable_data_json = json.load(f)
f.close()

AIRTABLE_URL = os.getenv("AIRTABLE_URL")
AIRTABLE_HEADER = {
    "Authorization": f"Bearer {os.getenv('AIRTABLE_TOKEN')}",
    "Content-Type": "application/json"
}
r = requests.post(url, headers=headers, json = airtable_data)
r.json()

if r.status_code == 201: #200好像也可以
    print("data has been succesfully uploaded")
else:
    raise Exception(
        f"Error occured when uploading, status_code: {r.status_code}"
    )
r = requests.post(url, headers=headers, json = airtable_data)
r.json()

In [ ]:
#formatting data to fit airtable api requiement
airtable_data = {'fields':{}}
for i in range(len(final_data)):
    airtable_data['fields']['black_box_query_id']:'tempId'
    airtable_data['black_box_query_asin']:'tempAsin'
    airtable_data['cerebro_query_id']:'tempCerebroId'
    airtable_data['phraseId']:'tempPhraseId'
    airtable_data['sponsoredPosition']:'tempSponsoredPosition'
    airtable_data['sponsoredScrapedAt']:'tempSponsoredScrapedAt'
    airtable_data['matchType']:'tempMatchType'
    airtable_data['organicPosition']:'tempOrganicPosition'
    airtable_data['organicScrapedAt']:'tempOrganicScrapedAt'
    airtable_data['phrase']:'tempPhrase'
    airtable_data['resultsNumber']:'tempResultsNumber'
    airtable_data['resultsNumberOver']:'tempResultsNumberOver'
    airtable_data['resultsNumberUpdatedAt']:'tempResultsNumberUpdatedAt'
    airtable_data['impressionExact30']:'tempImpressionExact30'
    airtable_data['exactTitleMatchProductsCount']:'tempExactTitleMatchProductsCount'
    airtable_data['iq']:'tempIq'
    airtable_data['sponsoredAsins']:'tempSponsoredAsins'
    airtable_data['amazonChoice']:'tempAmazonChoice'
    airtable_data['currentAsin']:'tempCurrentAsin'
    airtable_data['amzSuggestedPosition']:'tempAmzSuggestedPosition'
    airtable_data['amzSuggestedScrapedAt']:'tempAmzSuggestedScrapedAt'
    airtable_data['searchVolumeTrend30']:'tempSearchVolumeTrend30'
    airtable_data['newCprExact']:'tempNewCprExact'
    airtable_data['newCprBroad']:'tempNewCprBroad'
    airtable_data['monthlySales']:'tempMonthlySales'
    airtable_data['clickShareRate']:'tempClickShareRate'
    airtable_data['conversionShareRate']:'tempConversionShareRate'
    airtable_data['highCpc']: None
    airtable_data['lowCpc']: None
    airtable_data['searchFrequencyRank']: None
    airtable_data['octoberSeasonalSearchVolume']: None
    airtable_data['decemberSeasonalSearchVolume']: None
    airtable_data['novemberSeasonalSearchVolume']: None
    
  
    airtable_data['records'][i]['fields']['black_box_query_id']=final_data[i]['black_box_query_id']
    airtable_data['records'][i]['fields']['black_box_query_asin']=final_data[i]['black_box_query_asin']
    airtable_data['records'][i]['fields']['cerebro_query_id']=final_data[i]['cerebro_query_id']
    airtable_data['records'][i]['fields']['phraseId']=final_data[i]['phraseId']
    airtable_data['records'][i]['fields']['sponsoredPosition']=final_data[i]['sponsoredPosition']
    airtable_data['records'][i]['fields']['matchType']=final_data[i]['matchType']
    airtable_data['records'][i]['fields']['organicPosition']=final_data[i]['organicPosition']
    airtable_data['records'][i]['fields']['phrase']=final_data[i]['phrase']
    airtable_data['records'][i]['fields']['resultsNumber']=final_data[i]['resultsNumber']
    airtable_data['records'][i]['fields']['resultsNumberOver']=final_data[i]['resultsNumberOver']
    airtable_data['records'][i]['fields']['resultsNumberUpdatedAt']=final_data[i]['resultsNumberUpdatedAt']
    airtable_data['records'][i]['fields']['impressionExact30']=final_data[i]['impressionExact30']
    airtable_data['records'][i]['fields']['exactTitleMatchProductsCount']=final_data[i]['exactTitleMatchProductsCount']
    airtable_data['records'][i]['fields']['iq']=final_data[i]['iq']
    airtable_data['records'][i]['fields']['sponsoredAsins']=final_data[i]['sponsoredAsins']
    airtable_data['records'][i]['fields']['amazonChoice']=final_data[i]['amazonChoice']
    airtable_data['records'][i]['fields']['currentAsin']=final_data[i]['currentAsin']
    airtable_data['records'][i]['fields']['amzSuggestedPosition']=final_data[i]['amzSuggestedPosition']
    airtable_data['records'][i]['fields']['amzSuggestedScrapedAt']=final_data[i]['amzSuggestedScrapedAt']
    airtable_data['records'][i]['fields']['searchVolumeTrend30']=final_data[i]['searchVolumeTrend30']
    airtable_data['records'][i]['fields']['newCprExact']=final_data[i]['newCprExact']
    airtable_data['records'][i]['fields']['newCprBroad']=final_data[i]['newCprBroad']
    airtable_data['records'][i]['fields']['monthlySales']=final_data[i]['monthlySales']
    airtable_data['records'][i]['fields']['clickShareRate']=final_data[i]['clickShareRate']
    airtable_data['records'][i]['fields']['conversionShareRate']=final_data[i]['conversionShareRate']
    airtable_data['records'][i]['fields']['cpc']=final_data[i]['cpc']
    airtable_data['records'][i]['fields']['highCpc']=final_data[i]['highCpc']
    airtable_data['records'][i]['fields']['lowCpc']=final_data[i]['lowCpc']
    airtable_data['records'][i]['fields']['searchFrequencyRank']=final_data[i]['searchFrequencyRank']
    airtable_data['records'][i]['fields']['octoberSeasonalSearchVolume']=final_data[i]['octoberSeasonalSearchVolume']
    airtable_data['records'][i]['fields']['decemberSeasonalSearchVolume']=final_data[i]['decemberSeasonalSearchVolume']
    airtable_data['records'][i]['fields']['novemberSeasonalSearchVolume']=final_data[i]['novemberSeasonalSearchVolume']
    
    

    # dump as json
    # 待補

In [ ]:
#formatting data to fit airtable api requiement
airtable_data = {'records':[]}
for i in range(len(final_data)):
    airtable_data['records'].append({'fields':{}
    })

    airtable_data['records'][i]['fields']['black_box_query_id']=final_data[i]['black_box_query_id']
    airtable_data['records'][i]['fields']['black_box_query_asin']=final_data[i]['black_box_query_asin']
    airtable_data['records'][i]['fields']['cerebro_query_id']=final_data[i]['cerebro_query_id']
    airtable_data['records'][i]['fields']['phraseId']=final_data[i]['phraseId']
    airtable_data['records'][i]['fields']['sponsoredPosition']=final_data[i]['sponsoredPosition']
    airtable_data['records'][i]['fields']['matchType']=final_data[i]['matchType'][0]##
    airtable_data['records'][i]['fields']['organicPosition']=final_data[i]['organicPosition']
    airtable_data['records'][i]['fields']['phrase']=final_data[i]['phrase']
    airtable_data['records'][i]['fields']['resultsNumber']=final_data[i]['resultsNumber']
    airtable_data['records'][i]['fields']['resultsNumberOver']=final_data[i]['resultsNumberOver']
    airtable_data['records'][i]['fields']['resultsNumberUpdatedAt']=final_data[i]['resultsNumberUpdatedAt']
    airtable_data['records'][i]['fields']['impressionExact30']=final_data[i]['impressionExact30']
    airtable_data['records'][i]['fields']['exactTitleMatchProductsCount']=final_data[i]['exactTitleMatchProductsCount']
    airtable_data['records'][i]['fields']['iq']=final_data[i]['iq']
    airtable_data['records'][i]['fields']['sponsoredAsins']=final_data[i]['sponsoredAsins']
    airtable_data['records'][i]['fields']['amazonChoice']=final_data[i]['amazonChoice']
    airtable_data['records'][i]['fields']['currentAsin']=final_data[i]['currentAsin']
    airtable_data['records'][i]['fields']['amzSuggestedPosition']=final_data[i]['amzSuggestedPosition']
    airtable_data['records'][i]['fields']['amzSuggestedScrapedAt']=final_data[i]['amzSuggestedScrapedAt']
    airtable_data['records'][i]['fields']['searchVolumeTrend30']=final_data[i]['searchVolumeTrend30']
    airtable_data['records'][i]['fields']['newCprExact']=final_data[i]['newCprExact']
    airtable_data['records'][i]['fields']['newCprBroad']=final_data[i]['newCprBroad']
    airtable_data['records'][i]['fields']['monthlySales']=final_data[i]['monthlySales']
    airtable_data['records'][i]['fields']['clickShareRate']=final_data[i]['clickShareRate']
    airtable_data['records'][i]['fields']['conversionShareRate']=final_data[i]['conversionShareRate']
    airtable_data['records'][i]['fields']['cpc']=final_data[i]['cpc']
    airtable_data['records'][i]['fields']['highCpc']=final_data[i]['highCpc']
    airtable_data['records'][i]['fields']['lowCpc']=final_data[i]['lowCpc']
    airtable_data['records'][i]['fields']['searchFrequencyRank']=final_data[i]['searchFrequencyRank']
    airtable_data['records'][i]['fields']['octoberSeasonalSearchVolume']=final_data[i]['octoberSeasonalSearchVolume']
    airtable_data['records'][i]['fields']['decemberSeasonalSearchVolume']=final_data[i]['decemberSeasonalSearchVolume']
    airtable_data['records'][i]['fields']['novemberSeasonalSearchVolume']=final_data[i]['novemberSeasonalSearchVolume']

AIRTABLE_URL = os.getenv("AIRTABLE_URL")
AIRTABLE_HEADER = {
    "Authorization": f"Bearer {os.getenv('AIRTABLE_TOKEN')}",
    "Content-Type": "application/json"
}

for dictionary in airtable_data['records']:
    r = requests.post(url, headers=headers, json = dictionary)
    r.json()
    print(r.status_code)


    

NameError: name 'final_data' is not defined

In [ ]:
final_data[1]['matchType'][0]

'ppc'

In [ ]:
airtable_data


{'records': [{'fields': {'black_box_query_id': 9138985,
    'black_box_query_asin': 'B0088YVPN6'}},
  {'fields': {'black_box_query_id': 9138985,
    'black_box_query_asin': 'B0088YVPN6'}},
  {'fields': {'black_box_query_id': 9138985,
    'black_box_query_asin': 'B0088YVPN6'}},
  {'fields': {'black_box_query_id': 9138985,
    'black_box_query_asin': 'B0088YVPN6'}},
  {'fields': {'black_box_query_id': 9138985,
    'black_box_query_asin': 'B0088YVPN6'}},
  {'fields': {'black_box_query_id': 9138985,
    'black_box_query_asin': 'B0088YVPN6'}},
  {'fields': {'black_box_query_id': 9138985,
    'black_box_query_asin': 'B0088YVPN6'}},
  {'fields': {'black_box_query_id': 9138985,
    'black_box_query_asin': 'B0088YVPN6'}},
  {'fields': {'black_box_query_id': 9138985,
    'black_box_query_asin': 'B0088YVPN6'}},
  {'fields': {'black_box_query_id': 9138985,
    'black_box_query_asin': 'B0088YVPN6'}},
  {'fields': {'black_box_query_id': 9138985,
    'black_box_query_asin': 'B0088YVPN6'}},
  {'fields

In [ ]:
from helium10Scrapper.path import PROJECT_ROOT
import requests
import json
import pandas as pd
import time



BEARER_TOKEN = env.BEARER_TOKEN
ACCOUNT_ID = 


def create_black_box_query(the_filter):
    r = requests.post(
        f"https://research-tools.helium10.com/api/blackbox/v1/search/products?accountId={ACCOUNT_ID}",
        json=the_filter,  #the filter是black box filter 01.json
        headers={"Authorization": f"Bearer {BEARER_TOKEN}"},
    )

    black_box_query_creation_response = {}
    if r.status_code == 201:
        black_box_query_creation_response = r.json()
    else:
        raise Exception(
            f"Error occured when creating black box query, status_code: {r.status_code}"
        )

    return black_box_query_creation_response


def get_black_box_query_result(black_box_query_id):
    r = requests.get(
        f"https://research-tools.helium10.com/api/blackbox/v1/search/products/{black_box_query_id}/results?",
        params={"accountId": ACCOUNT_ID, "page": 1, "per_page": 1000},
        headers={"Authorization": f"Bearer {BEARER_TOKEN}"},
    )
    black_box_result_dict = r.json()
    return black_box_result_dict


def create_cerebro_query(product_asin):
    cerebro_create_query_endpoint = (
        f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single"
    )
    params = {
        "accountId": ACCOUNT_ID,
    }
    body = {
        "marketplace": "ATVPDKIKX0DER",
        "productId": product_asin,
        "adminSearch": False,
        "exactProduct": False,
    }
    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    r = requests.post(
        url=cerebro_create_query_endpoint, params=params, headers=headers, json=body
    )
    return r


def get_cerebro_query_result(cerebro_query_id):
    cerebro_get_query_endpoint = f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single/{cerebro_query_id}/data"
    
    params = {
        "accountId": ACCOUNT_ID,
        "include-all": 0,
        "include-any": 1,
        "page": 1,
        "per_page": 1000,
        "sort": "-iq",
    }

    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    r = requests.get(cerebro_get_query_endpoint, params=params, headers=headers)
    return r


if __name__ == "__main__":
    BATCH_ID = "9999"
    # get filter from pre-written json file
    f = open(PROJECT_ROOT.joinpath("black_box_filter.01.json")) #之後這邊要補上遍歷各個filter的程式碼Z
    all_cat_filter = json.load(f)
    f.close()

    # create black box query
    print("create black box query...")
    black_box_query_creation_response = create_black_box_query(all_cat_filter)  #response裡面是字典裝字典 第二個字典裡項目是key 值是資料內容

    # parse black box query_id
    print("parse black box query...")

    black_box_query_id = ""
    try:
        black_box_query_id = black_box_query_creation_response["data"]["id"]
    except KeyError:
        raise (
            "No id found in response, error may occrued when creating black box query"
        )

    # get black box query result
    print("get black box result")
    black_box_query_id = black_box_query_creation_response["data"]["id"]
    black_box_result_dict = get_black_box_query_result(
        black_box_query_id=black_box_query_id  #***********************
    )

    black_box_result_data = []
    try:
        black_box_result_data = black_box_result_dict["data"]["tableData"]   #有一大堆商品，然後每個商品都有id跟asin。
    except KeyError:
        raise (
            "Can't find table data, Error occrued when requesting for black box query result"
        )

    # dump resutl to csv
    print("dump black box result")
    data = []
    for data_dict in black_box_result_data:
        try:
            data.append([black_box_query_id, data_dict["asin"], data_dict["id"]])
        except KeyError:
            continue

    black_box_result_tb = pd.DataFrame(data, columns=["query_id", "asin", "id"])
    black_box_result_tb.to_csv(
        PROJECT_ROOT.joinpath(f"black_box_result.{BATCH_ID}.csv"), index=False
    )

    # create cerebro query for each asin in black_box_result_data
    print("create cerebro query result")
    cerebro_query_id_dict = {}
    for product in black_box_result_data[:10]:
        asin = product["asin"]
        print(asin, end="..., ")
        r = create_cerebro_query(asin)
        print(r.status_code, end=", sleeping...\n")
        cerebro_query_id_dict[asin] = r 
        time.sleep(5)

    # dump cerebro result as json
    print("dump cerebro query response (as json)")
    output_tmp_dict = {}
    for asin, response in cerebro_query_id_dict.items():
        output_tmp_dict[asin] = response.json()

    f = open(PROJECT_ROOT.joinpath(f"cerebro_query_id_dict.{BATCH_ID}.json"), "w")
    json.dump(output_tmp_dict, f, indent=4)
    f.close()

    # get cerebro query result
    print("get cerebro query result")
    cerebro_result_dict = {}
    cerebro_query_id_list = [] ################ new
    for asin, response in cerebro_query_id_dict.items():
        print(asin, end="\r")
        cerebro_query_id = ""
        
        try:
            cerebro_query_id = response.json()["data"]["id"]
            cerebro_query_id_list.append(cerebro_query_id) ############### new
        except KeyError:
            print("no result for this asin")
            continue

        cerebro_result_dict[asin] = get_cerebro_query_result(cerebro_query_id)
        print(cerebro_result_dict[asin].status_code, end="\r")
        time.sleep(5)



    # merging data
    final_data = []      
    for black_box_asin in cerebro_result_dict.keys() : 
        final_data = cerebro_result_dict[black_box_asin].json()['data']['tableData'].copy()
        for i in range(len(final_data)):
            final_data[i]['black_box_query_id'] =black_box_query_id #filter 01 的 black box id 
            final_data[i]['black_box_query_asin']=black_box_asin
            final_data[i]['cerebro_query_id']= cerebro_query_id_dict[black_box_asin].json()['data']['id']

    
    #formatting data to fit airtable api requiement
    airtable_data = {'records':[]}
    for i in range(len(final_data)):
        airtable_data['records'].append({'fields':{}
        })

        airtable_data['records'][i]['fields']['black_box_query_id']=final_data[i]['black_box_query_id']
        airtable_data['records'][i]['fields']['black_box_query_asin']=final_data[i]['black_box_query_asin']
        airtable_data['records'][i]['fields']['cerebro_query_id']=final_data[i]['cerebro_query_id']
        airtable_data['records'][i]['fields']['phraseId']=final_data[i]['phraseId']
        airtable_data['records'][i]['fields']['sponsoredPosition']=final_data[i]['sponsoredPosition']
        airtable_data['records'][i]['fields']['matchType']=final_data[i]['matchType'][0]##
        airtable_data['records'][i]['fields']['organicPosition']=final_data[i]['organicPosition']
        airtable_data['records'][i]['fields']['phrase']=final_data[i]['phrase']
        airtable_data['records'][i]['fields']['resultsNumber']=final_data[i]['resultsNumber']
        airtable_data['records'][i]['fields']['resultsNumberOver']=final_data[i]['resultsNumberOver']
        airtable_data['records'][i]['fields']['resultsNumberUpdatedAt']=final_data[i]['resultsNumberUpdatedAt']
        airtable_data['records'][i]['fields']['impressionExact30']=final_data[i]['impressionExact30']
        airtable_data['records'][i]['fields']['exactTitleMatchProductsCount']=final_data[i]['exactTitleMatchProductsCount']
        airtable_data['records'][i]['fields']['iq']=final_data[i]['iq']
        airtable_data['records'][i]['fields']['sponsoredAsins']=final_data[i]['sponsoredAsins']
        airtable_data['records'][i]['fields']['amazonChoice']=final_data[i]['amazonChoice']
        airtable_data['records'][i]['fields']['currentAsin']=final_data[i]['currentAsin']
        airtable_data['records'][i]['fields']['amzSuggestedPosition']=final_data[i]['amzSuggestedPosition']
        airtable_data['records'][i]['fields']['amzSuggestedScrapedAt']=final_data[i]['amzSuggestedScrapedAt']
        airtable_data['records'][i]['fields']['searchVolumeTrend30']=final_data[i]['searchVolumeTrend30']
        airtable_data['records'][i]['fields']['newCprExact']=final_data[i]['newCprExact']
        airtable_data['records'][i]['fields']['newCprBroad']=final_data[i]['newCprBroad']
        airtable_data['records'][i]['fields']['monthlySales']=final_data[i]['monthlySales']
        airtable_data['records'][i]['fields']['clickShareRate']=final_data[i]['clickShareRate']
        airtable_data['records'][i]['fields']['conversionShareRate']=final_data[i]['conversionShareRate']
        airtable_data['records'][i]['fields']['cpc']=final_data[i]['cpc']
        airtable_data['records'][i]['fields']['highCpc']=final_data[i]['highCpc']
        airtable_data['records'][i]['fields']['lowCpc']=final_data[i]['lowCpc']
        airtable_data['records'][i]['fields']['searchFrequencyRank']=final_data[i]['searchFrequencyRank']
        airtable_data['records'][i]['fields']['octoberSeasonalSearchVolume']=final_data[i]['octoberSeasonalSearchVolume']
        airtable_data['records'][i]['fields']['decemberSeasonalSearchVolume']=final_data[i]['decemberSeasonalSearchVolume']
        airtable_data['records'][i]['fields']['novemberSeasonalSearchVolume']=final_data[i]['novemberSeasonalSearchVolume']

    url =env.url
AIRTABLE_HEADER = {
    "Authorization": f"Bearer {os.getenv('AIRTABLE_TOKEN')}",
    "Content-Type": "application/json"
}

    for dictionary in airtable_data['records']:
        r = requests.post(url, headers=headers, json = dictionary)
        r.json()
        print(r.status_code)
    
    

    # dump as json
    f = open(PROJECT_ROOT.joinpath(f"cerebro_query_result.{BATCH_ID}.json"), "w")
    json.dump(airtable_data, f, indent=4)
    f.close()


: 

In [ ]:
AIRTABLE_URL = os.getenv("AIRTABLE_URL") AIRTABLE_HEADER = { "Authorization": f"Bearer {os.getenv('AIRTABLE_TOKEN')}", "Content-Type": "application/json" }

In [ ]:


AIRTABLE_URL = os.getenv("AIRTABLE_URL")
AIRTABLE_HEADER = {
    "Authorization": f"Bearer {os.getenv('AIRTABLE_TOKEN')}",
    "Content-Type": "application/json"
}

headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}

response = requests.get("https://api.airtable.com/v0/",headers=AIRTABLE_HEADER)


table_recod_id = []
for i in response.json().get('records'):
    id = i.get('id')
    table_recod_id.append(id)

for record in table_recod_id:
    r = requests.delete(AIRTABLE_URL + '/' + record, headers=AIRTABLE_HEADER)

In [ ]:
from helium10Scrapper.path import PROJECT_ROOT
import requests
import json
import pandas as pd
import time





def create_black_box_query(the_filter):
    r = requests.post(
        f"https://research-tools.helium10.com/api/blackbox/v1/search/products?accountId={ACCOUNT_ID}",
        json=the_filter,  #the filter是black box filter 01.json
        headers={"Authorization": f"Bearer {BEARER_TOKEN}"},
    )

    black_box_query_creation_response = {}
    if r.status_code == 201:
        black_box_query_creation_response = r.json()
    else:
        raise Exception(
            f"Error occured when creating black box query, status_code: {r.status_code}"
        )

    return black_box_query_creation_response


def get_black_box_query_result(black_box_query_id):
    r = requests.get(
        f"https://research-tools.helium10.com/api/blackbox/v1/search/products/{black_box_query_id}/results?",
        params={"accountId": ACCOUNT_ID, "page": 1, "per_page": 1000},
        headers={"Authorization": f"Bearer {BEARER_TOKEN}"},
    )
    black_box_result_dict = r.json()
    return black_box_result_dict


def create_cerebro_query(product_asin):
    cerebro_create_query_endpoint = (
        f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single"
    )
    params = {
        "accountId": ACCOUNT_ID,
    }
    body = {
        "marketplace": "ATVPDKIKX0DER",
        "productId": product_asin,
        "adminSearch": False,
        "exactProduct": False,
    }
    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    r = requests.post(
        url=cerebro_create_query_endpoint, params=params, headers=headers, json=body
    )
    return r


def get_cerebro_query_result(cerebro_query_id):
    cerebro_get_query_endpoint = f"https://research-tools.helium10.com/api/cerebro/v1/amazon/search/single/{cerebro_query_id}/data"
    
    params = {
        "accountId": ACCOUNT_ID,
        "include-all": 0,
        "include-any": 1,
        "page": 1,
        "per_page": 1000,
        "sort": "-iq",
    }

    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    r = requests.get(cerebro_get_query_endpoint, params=params, headers=headers)
    return r









if __name__ == "__main__":
    BATCH_ID = "9999"
    # get filter from pre-written json file
    f = open(PROJECT_ROOT.joinpath("black_box_filter.01.json")) #之後這邊要補上遍歷各個filter的程式碼Z
    all_cat_filter = json.load(f)
    f.close()

    # create black box query
    print("create black box query...")
    black_box_query_creation_response = create_black_box_query(all_cat_filter)  #response裡面是字典裝字典 第二個字典裡項目是key 值是資料內容

    # parse black box query_id
    print("parse black box query...")

    black_box_query_id = ""
    try:
        black_box_query_id = black_box_query_creation_response["data"]["id"]
    except KeyError:
        raise (
            "No id found in response, error may occrued when creating black box query"
        )

    # get black box query result
    print("get black box result")
    black_box_query_id = black_box_query_creation_response["data"]["id"]
    black_box_result_dict = get_black_box_query_result(
        black_box_query_id=black_box_query_id  #***********************
    )

    black_box_result_data = []
    try:
        black_box_result_data = black_box_result_dict["data"]["tableData"]   #有一大堆商品，然後每個商品都有id跟asin。
    except KeyError:
        raise (
            "Can't find table data, Error occrued when requesting for black box query result"
        )

    # dump resutl to csv
    print("dump black box result")
    data = []
    for data_dict in black_box_result_data:
        try:
            data.append([black_box_query_id, data_dict["asin"], data_dict["id"]])
        except KeyError:
            continue

    black_box_result_tb = pd.DataFrame(data, columns=["query_id", "asin", "id"])
    black_box_result_tb.to_csv(
        PROJECT_ROOT.joinpath(f"black_box_result.{BATCH_ID}.csv"), index=False
    )

    # create cerebro query for each asin in black_box_result_data
    print("create cerebro query result")
    cerebro_query_id_dict = {}
    for product in black_box_result_data[:10]:
        asin = product["asin"]
        print(asin, end="..., ")
        r = create_cerebro_query(asin)
        print(r.status_code, end=", sleeping...\n")
        cerebro_query_id_dict[asin] = r 
        time.sleep(5)

    # dump cerebro result as json
    print("dump cerebro query response (as json)")
    output_tmp_dict = {}
    for asin, response in cerebro_query_id_dict.items():
        output_tmp_dict[asin] = response.json()

    f = open(PROJECT_ROOT.joinpath(f"cerebro_query_id_dict.{BATCH_ID}.json"), "w")
    json.dump(output_tmp_dict, f, indent=4)
    f.close()

    # get cerebro query result
    print("get cerebro query result")
    cerebro_result_dict = {}
    cerebro_query_id_list = [] ################ new
    for asin, response in cerebro_query_id_dict.items():
        print(asin, end="\r")
        cerebro_query_id = ""
        
        try:
            cerebro_query_id = response.json()["data"]["id"]
            cerebro_query_id_list.append(cerebro_query_id) ############### new
        except KeyError:
            print("no result for this asin")
            continue

        cerebro_result_dict[asin] = get_cerebro_query_result(cerebro_query_id)
        print(cerebro_result_dict[asin].status_code, end="\r")
        time.sleep(5)


create black box query...


parse black box query...
get black box result
dump black box result
create cerebro query result
B076T3N4RY..., 201, sleeping...
B01COPK6X4..., 201, sleeping...
B071NQ95CC..., 201, sleeping...
B01HMVA4XM..., 201, sleeping...
B01MS5QFVR..., 201, sleeping...
B007TG2M0E..., 201, sleeping...
B00JGRWA24..., 201, sleeping...
B00UG8D02G..., 201, sleeping...
B004EHZ820..., 201, sleeping...
B00I2STI4Q..., 201, sleeping...
dump cerebro query response (as json)
get cerebro query result


In [ ]:
airtable_data

{'records': [{'fields': {'black_box_query_id': 9209392,
    'black_box_query_asin': 'B00I2STI4Q',
    'cerebro_query_id': 122779753,
    'phraseId': 179332,
    'sponsoredPosition': None,
    'matchType': 'ppc',
    'organicPosition': None,
    'phrase': 'same day delivery',
    'resultsNumber': 1,
    'resultsNumberOver': False,
    'resultsNumberUpdatedAt': 1700793951,
    'impressionExact30': 2705,
    'exactTitleMatchProductsCount': 0,
    'iq': 2705000,
    'sponsoredAsins': 0,
    'amazonChoice': 'B0BZQ8NTQQ',
    'currentAsin': 'B00I2STI4Q',
    'amzSuggestedPosition': 56,
    'amzSuggestedScrapedAt': 1701256284,
    'searchVolumeTrend30': -2,
    'newCprExact': 27.44895,
    'newCprBroad': 27.44895,
    'monthlySales': 218,
    'clickShareRate': None,
    'conversionShareRate': None,
    'cpc': 117,
    'highCpc': 154,
    'lowCpc': 65,
    'searchFrequencyRank': None,
    'octoberSeasonalSearchVolume': 7438,
    'decemberSeasonalSearchVolume': 67835,
    'novemberSeasonalSearc